In [14]:
# !pip install openpyxl

In [192]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 234501/234501 [00:00<00:00, 981683.65it/s]


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,Healey,1720.007287,Spain,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,Partida,1518.782461,Spain,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,Spain,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,Pinkerton,1468.954354,Spain,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,Abernathy,1683.914742,France,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022


In [193]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [194]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 234501/234501 [00:00<00:00, 945578.00it/s]


In [195]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [196]:
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 234501/234501 [00:00<00:00, 1289680.63it/s]


In [200]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: 1 if x > 7 else 0)

100%|██████████| 234501/234501 [00:00<00:00, 1084711.30it/s]


In [202]:
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: 1 if x >= 3 else 0 )

100%|██████████| 234501/234501 [00:00<00:00, 1059877.55it/s]


In [203]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Card Type,Point Earned,date,City,month,year,Account_type,Other Account,Other Accounts,Salary Account
0,15572489,Healey,1720.0,Spain,Female,73,29,393987.727351,1,1,...,DIAMOND,2531.0,1-6-2022,Cedar Falls,6,2022,Gold,1,1,1
1,15800464,Partida,1519.0,Spain,Male,53,15,274502.817582,2,1,...,SILVER,1725.0,1-2-2022,Iowa City,2,2022,Premium,1,1,1
2,15580134,Crawford,503.0,Spain,Male,27,2,184535.891500,1,1,...,DIAMOND,711.0,1-1-2020,Council Bluffs,1,2020,Normal,0,0,1
3,15074588,Pinkerton,1469.0,Spain,Male,49,15,0.000000,1,1,...,SILVER,1512.0,1-1-2022,Iowa City,1,2022,Premium,1,1,1
4,15982065,Abernathy,1684.0,France,Male,49,30,0.000000,2,1,...,DIAMOND,1296.0,1-6-2022,Rio Rancho,6,2022,Gold,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234496,15244801,King,1638.0,France,Male,65,37,0.000000,1,1,...,GOLD,643.0,1-10-2022,Albuquerque,10,2022,Gold,1,1,1
234497,15047568,Medley,612.0,France,Male,32,1,172743.096102,1,0,...,DIAMOND,722.0,1-4-2020,Albuquerque,4,2020,Normal,0,0,0
234498,15617037,Wyman,856.0,Spain,Male,37,2,213987.894170,1,1,...,SILVER,870.0,1-10-2020,Sioux City,10,2020,Normal,0,0,1
234499,15788634,Romani,827.0,Spain,Female,37,2,124219.939284,1,0,...,DIAMOND,1018.0,1-2-2020,Cedar Rapids,2,2020,Normal,0,0,1


In [ ]:
df.to_csv("/data/")